# ScratchNet
Ein einfaches künstliches neuronales Netz mit einer von `keras` inspirierten API.
Hinweis: Das Netz wurde ausschließlich für Lernzwecke verfasst.

In [1]:
%tensorflow_version 2.x
import abc
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import random
import time
from tqdm import tqdm, trange

TensorFlow 2.x selected.


In [0]:
class DifferentiableFunction(abc.ABC):
    def derivative(self, net_input):
        pass

    def __call__(self, net_input):
        pass

In [0]:
class Sigmoid(DifferentiableFunction):
    def derivative(self, net_input):
        return self(net_input) * (1 - self(net_input))

    def __call__(self, net_input):
        return 1 / (1 + np.exp(-net_input))

In [0]:
class SquaredError(DifferentiableFunction):
    def derivative(self, target, actual):
        return actual - target

    def __call__(self, target, actual):
        return 0.5 * np.sum((target - actual) ** 2)

In [0]:
class DenseLayer:
    def __init__(
        self,
        neuron_count,
        depth=None,
        activation=None,
        biases=None,
        weights=None,
        prev_layer=None,
        next_layer=None,
    ):
        self.depth = depth
        self.next_layer = next_layer
        self.prev_layer = prev_layer

        self.neuron_count = neuron_count
        self.activation_func = activation or Sigmoid()

        self.weights = weights
        self.biases = biases

    def prepare_inputs(self, images, labels=None):
        return images if labels is None else images, labels

    def initialize_parameters(self):
        if self.weights is None:
            self.weights = np.random.randn(
                self.neuron_count, self.prev_layer.neuron_count
            )
        if self.biases is None:
            self.biases = np.random.randn(self.neuron_count, 1)

    def compute_cost_gradients(self, label_vec, cost_func):
        cost_gradients = cost_func.derivative(
            self.activation_vec, label_vec
        ) * self.activation_func.derivative(self.layer_inputs)
        self._update_layer_gradients(cost_gradients)
        return cost_gradients

    def feed_backwards(self, prev_input_gradients):
        new_input_gradients = np.dot(
            self.next_layer.weights.transpose(), prev_input_gradients
        ) * self.activation_func.derivative(self.layer_inputs)
        self._update_layer_gradients(new_input_gradients)
        return new_input_gradients

    def _update_layer_gradients(self, input_gradients):
        self.bias_gradients = input_gradients
        self.weight_gradients = np.dot(
            input_gradients, self.prev_layer.activation_vec.transpose()
        )

    def feed_forward_layer(self, input_activations):
        self.layer_inputs = np.dot(
            self.weights, input_activations) + self.biases
        self.activation_vec = self.activation_func(self.layer_inputs)
        return self.activation_vec

    def inspect(self):
        print(f"--------- Layer L={self.depth} ---------")
        print(f"  # Neuronen: {self.neuron_count}")
        for n in range(self.neuron_count):
            print(f"    Neuron {n}")
            if self.prev_layer:
                for w in self.weights[n]:
                    print(f"      Weight: {w}")
                print(f"      Bias: {self.biases[n][0]}")

In [0]:
class FlattenLayer(DenseLayer):
    def __init__(self, input_shape):
        total_input_neurons = 1
        for dim in input_shape:
            total_input_neurons *= dim
        super().__init__(neuron_count=total_input_neurons)

    def initialize_parameters(self):
        pass

    def feed_forward_layer(self, input_activations):
        self.activation_vec = input_activations
        return input_activations

    def prepare_inputs(self, images, labels=None):
        flattened_images = images.reshape(
            images.shape[0], self.neuron_count, 1)
        if labels is not None:
            labels = labels.reshape(labels.shape[0], -1, 1)
            return flattened_images, labels
        return flattened_images

In [0]:
class ScratchNet:
    def __init__(self, layers):
        self.learning_rate = 0.5
        self.cost_func = SquaredError()
        self.layers = layers
        for index, layer in enumerate(self.layers):
            layer.prev_layer = self.layers[index - 1] if index > 0 else None
            layer.next_layer = (
                self.layers[index + 1] if index +
                1 < len(self.layers) else None
            )
            layer.depth = index
            layer.initialize_parameters()

    def _calculate_loss(self, input_samples):
        total_error = 0.0
        for sample in input_samples:
            image, label_vec = sample
            output_activations = self._feed_forward(image)
            total_error += self.cost_func(label_vec, output_activations)
        return total_error / len(input_samples)

    def _calculate_accuracy(self, input_samples):
        results = [
            (np.argmax(self._feed_forward(image)), np.argmax(expected_label))
            for image, expected_label in input_samples
        ]
        num_correct = sum(int(x == y) for (x, y) in results)
        return num_correct / len(input_samples)

    def _feed_forward(self, input_sample):
        for layer in self.layers:
            input_sample = layer.feed_forward_layer(input_sample)
        return input_sample

    def _update_parameters(self, input_samples):
        weight_gradients = [np.zeros(layer.weights.shape)
                            for layer in self.layers[1:]]
        bias_gradients = [np.zeros(layer.biases.shape)
                          for layer in self.layers[1:]]

        for sample in input_samples:
            sample_weight_gradients, sample_bias_gradients = self._backpropagate(
                sample)
            weight_gradients = np.add(
                weight_gradients, sample_weight_gradients)
            bias_gradients = np.add(bias_gradients, sample_bias_gradients)

        for layer, layer_weight_gradients, layer_bias_gradients in zip(
            self.layers[1:], weight_gradients, bias_gradients
        ):
            layer.weights += (
                self.learning_rate *
                layer_weight_gradients / len(input_samples)
            )
            layer.biases += (
                self.learning_rate * layer_bias_gradients / len(input_samples)
            )

    def _backpropagate(self, training_sample):
        train_input, train_output = training_sample
        self._feed_forward(train_input)
        gradients = self.layers[-1].compute_cost_gradients(
            train_output, cost_func=self.cost_func
        )

        for layer in reversed(self.layers[1:-1]):
            gradients = layer.feed_backwards(gradients)

        weight_gradients = [
            layer.weight_gradients for layer in self.layers[1:]]
        bias_gradients = [layer.bias_gradients for layer in self.layers[1:]]
        return weight_gradients, bias_gradients

    def _stochastic_gradient_descent(
        self, training_data, epochs=1, batch_size=1, avg_lookbehind=None
    ):
        losses, accuracies = list(), list()
        training_set_size = len(training_data)

        avg_lookbehind = avg_lookbehind or int(
            0.10 * training_set_size / batch_size)
        running_loss, running_acc = [], []

        for epoch in range(epochs):
            random.shuffle(training_data)

            with tqdm(total=training_set_size) as progress:
                for t in range(0, training_set_size, batch_size):
                    batch = training_data[t: t + batch_size]

                    self._update_parameters(batch)
                    loss, accuracy = (
                        self._calculate_loss(batch),
                        self._calculate_accuracy(batch),
                    )

                    running_loss = ([loss] + running_loss)[:avg_lookbehind]
                    running_acc = ([accuracy] + running_acc)[:avg_lookbehind]
                    running_loss_avg = np.average(running_loss)
                    running_acc_avg = np.average(running_acc)
                    losses.append(running_loss_avg)
                    accuracies.append(running_acc_avg)

                    progress.set_description(f"Epoch {epoch+1}")
                    progress.set_postfix(
                        loss="{0:.3f}".format(running_loss_avg),
                        accuracy="{0:.2f}".format(running_acc_avg),
                    )

                    progress.update(len(batch))
        return losses, accuracies

    def fit(self, train_images, train_labels, epochs=1, batch_size=1):
        train_images, train_labels = self.layers[0].prepare_inputs(
            train_images, train_labels
        )
        training_data = list(zip(train_images, train_labels))
        losses, accuracies = self._stochastic_gradient_descent(
            training_data, epochs=epochs, batch_size=batch_size
        )
        return losses, accuracies

    def predict(self, model_inputs):
        model_inputs = self.layers[0].prepare_inputs(model_inputs)
        predicted = np.zeros(
            (model_inputs.shape[0], self.layers[-1].neuron_count, 1))
        for i, model_input in enumerate(model_inputs):
            predicted[i] = self._feed_forward(model_input)
        return predicted

    def evaluate(self, validation_images, validation_labels):
        validation_images, validation_labels = self.layers[0].prepare_inputs(
            validation_images, validation_labels
        )
        validation_data = list(zip(validation_images, validation_labels))
        return (
            self._calculate_loss(validation_data),
            self._calculate_accuracy(validation_data),
        )

    def compile(self, learning_rate=None, loss=None):
        self.learning_rate = learning_rate or self.learning_rate
        self.cost_func = loss or self.cost_func

    def inspect(self):
        print(f"--------- {self.__class__.__name__} ---------")
        print(f"  # Inputs: {self.layers[0].neuron_count}")
        for layer in self.layers:
            layer.inspect()